In [1]:
from sqlalchemy import create_engine
import pandas as pd

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
query = '''
SELECT facility, SUM(revenue) AS total_revenue
FROM (
    SELECT name AS facility, SUM(slots * membercost) AS revenue
    FROM Bookings
    JOIN Members ON Members.memid = Bookings.memid
    JOIN Facilities ON Facilities.facid = Bookings.facid
    WHERE Bookings.memid <> 0
    GROUP BY Facilities.facid
    
    UNION ALL
    
    SELECT name AS facility, SUM(slots * guestcost) AS revenue
    FROM Bookings
    JOIN Members ON Members.memid = Bookings.memid
    JOIN Facilities ON Facilities.facid = Bookings.facid
    WHERE Bookings.memid = 0
    GROUP BY Facilities.facid
) AS subquery
GROUP BY facility
HAVING total_revenue < 1000
ORDER BY total_revenue DESC;
'''

result = engine.execute(query)
df = pd.DataFrame(result.fetchall(), columns=['facility', 'total_revenue'])
df

,facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [3]:
query = '''
SELECT m.firstname || ' ' || m.surname AS member, r.firstname || ' ' || r.surname AS recommender
FROM Members m
LEFT JOIN Members r ON m.recommendedby = r.memid
ORDER BY m.surname, m.firstname;
'''
result = engine.execute(query)
df = pd.DataFrame(result.fetchall(), columns=['member', 'recommender'])
df

,member,recommender
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith
5,Joan Coplin,Timothy Baker
6,Erica Crumpet,Tracy Smith
7,Nancy Dare,Janice Joplette
8,David Farrell,None
9,Jemima Farrell,None


In [4]:
query = '''
SELECT Facilities.name AS facility, SUM(Bookings.slots)*0.5 AS hours
FROM Bookings
JOIN Members ON Members.memid = Bookings.memid
JOIN Facilities ON Facilities.facid = Bookings.facid
WHERE Members.memid > 0
GROUP BY Facilities.name;
'''
result = engine.execute(query)
df = pd.DataFrame(result.fetchall(), columns=['facility', 'hours'])
df

,facility,hours
0,Badminton Court,543.0
1,Massage Room 1,442.0
2,Massage Room 2,27.0
3,Pool Table,428.0
4,Snooker Table,430.0
5,Squash Court,209.0
6,Table Tennis,397.0
7,Tennis Court 1,478.5
8,Tennis Court 2,441.0


Q13: Find the facilities usage by month, but not guests

In [5]:
query = '''
SELECT MIN(starttime) AS firstdate, MAX(starttime) AS lastdate
FROM Bookings;
'''
result = engine.execute(query)
df = pd.DataFrame(result.fetchall(), columns=['firstdate', 'lastdate'])
df

,firstdate,lastdate
0,2012-07-03 08:00:00,2012-09-30 19:30:00


The dataset covers records from July to September, 3 momths.

In [6]:
query = '''
SELECT Facilities.name AS facility, Julyp.July, Augustp.August, Septemberp.September
FROM Facilities
LEFT JOIN (
    SELECT Facilities.facid, SUM(Bookings.slots)*0.5 AS July
    FROM Bookings
    JOIN Members ON Members.memid = Bookings.memid
    JOIN Facilities ON Facilities.facid = Bookings.facid
    WHERE Members.memid > 0 AND starttime LIKE '2012-07%'
    GROUP BY Facilities.facid
) AS Julyp ON Facilities.facid = Julyp.facid
LEFT JOIN (
    SELECT Facilities.facid, SUM(Bookings.slots)*0.5 AS August
    FROM Bookings
    JOIN Members ON Members.memid = Bookings.memid
    JOIN Facilities ON Facilities.facid = Bookings.facid
    WHERE Members.memid > 0 AND starttime LIKE '2012-08%'
    GROUP BY Facilities.facid
) AS Augustp ON Facilities.facid = Augustp.facid
LEFT JOIN (
    SELECT Facilities.facid, SUM(Bookings.slots)*0.5 AS September
    FROM Bookings
    JOIN Members ON Members.memid = Bookings.memid
    JOIN Facilities ON Facilities.facid = Bookings.facid
    WHERE Members.memid > 0 AND starttime LIKE '2012-09%'
    GROUP BY Facilities.facid
) AS Septemberp ON Facilities.facid = Septemberp.facid;
'''
result = engine.execute(query)
df = pd.DataFrame(result.fetchall(), columns=['facility', 'July', 'August', 'September'])
df

,facility,July,August,September
0,Tennis Court 1,100.5,169.5,208.5
1,Tennis Court 2,61.5,172.5,207.0
2,Badminton Court,82.5,207.0,253.5
3,Table Tennis,49.0,148.0,200.0
4,Massage Room 1,83.0,158.0,201.0
5,Massage Room 2,4.0,9.0,14.0
6,Squash Court,25.0,92.0,92.0
7,Snooker Table,70.0,158.0,202.0
8,Pool Table,55.0,151.5,221.5
